In this notebook we verify the data from our table. I.e. we prove that each pair of knots in the tables share the same 0-surgeries and are not isotopic.

In [1]:
import snappy
import regina
import csv
import time

### FINDING POSITIVE TRIANGULATIONS (This is a slight adaption of Dunfield's code) 
### --> NEEDED FOR MANY OTHERS

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations


def better_find_positive_triangulation(M,tries=1):
    '''
    Search for a positive triangulation, but ignores errors.
    '''
    RandomizeCount=0
    while RandomizeCount<tries:
        try:
            X=find_positive_triangulation(M)
            return X
        except snappy.SnapPeaFatalError:
            M.randomize()
            RandomizeCount=RandomizeCount+1
    return None

### COMPUTING THE VOLUME

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0


### CHECKING FOR ISOMETRIES

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """ 
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []
    
### CHECKING FOR A COMBINATORIAL EQUIVALENCE WITH REGINA    

def to_regina(data):
    '''
    This function was written by Dunfield.
    It sends a SnapPy triangulation to regina.
    '''
    if hasattr(data, '_to_string'):
        data = data._to_string()
    if isinstance(data, str):
        if data.find('(') > -1:
            data = closed_isosigs(data)[0]
        return regina.Triangulation3(data)
    assert isinstance(data, regina.Triangulation3)
    return data

def display1(sig, tri):
    signatures1.append(sig)
    return False

def display2(sig, tri):
    signatures2.append(sig)
    return False

def search_for_combinatorial_equivalence(M,N,index_M=1,index_N=1):
    '''
    Takes as input two snappy triangulations, transforms them to regina and searches through the pachner graph for an 
    equivalence.
    '''
    F1=M.filled_triangulation()
    F2=N.filled_triangulation()
    T1=to_regina(F1)
    T2=to_regina(F2)
    T1.simplifyExhaustive()
    T2.simplifyExhaustive()
    global signatures1
    signatures1=[]
    global signatures2
    signatures2=[]
    T1.retriangulate(index_M,1,display1)
    T2.retriangulate(index_N,1,display2)
    for s1 in signatures1:
        for s2 in signatures2:
            if s1==s2:
                return True
    return False

In [2]:
friends=[]

with open('census_friends.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friends.append([row[0],eval(row[1])])

with open('low_crossing_friends.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friends.append([row[0],eval(row[1])])
        
len(friends)

2050

In [3]:
start_time=time.time()

unclear_friends=[]
verified_friends=[]
for [knot,PD] in friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K)
    volF=better_volume(F)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if better_is_isometric_to(K,F)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time, 'seconds')

Verified friends: 2027
Unclear friends: 23
time taken: 246.10720205307007 seconds


In [4]:
start_time=time.time()

for [knot,PD] in unclear_friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K,try_hard=True)
    volF=better_volume(F,try_hard=True)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if better_is_isometric_to(K,F,try_hard=True)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time,'seconds')

Verified friends: 2041
Unclear friends: 9
time taken: 2.3918731212615967 seconds


In [5]:
start_time=time.time()

for [knot,PD] in unclear_friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K,try_hard=True)
    volF=better_volume(F,try_hard=True)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if better_is_isometric_to(K,F,try_hard=True)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time,'seconds')

Verified friends: 2041
Unclear friends: 9
time taken: 0.5751070976257324 seconds


In [6]:
start_time=time.time()

for [knot,PD] in unclear_friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K,try_hard=True)
    volF=better_volume(F,try_hard=True)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if better_is_isometric_to(K,F,try_hard=True)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time,'seconds')

Verified friends: 2041
Unclear friends: 9
time taken: 0.5746963024139404 seconds


In [7]:
start_time=time.time()

for [knot,PD] in unclear_friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K,try_hard=True)
    volF=better_volume(F,try_hard=True)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if search_for_combinatorial_equivalence(K,F,index_M=1,index_N=1)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time,'seconds')

Verified friends: 2048
Unclear friends: 2
time taken: 6.5163960456848145 seconds


In [8]:
start_time=time.time()

for [knot,PD] in unclear_friends:
    K=snappy.Manifold(knot)
    F=snappy.Link(PD).exterior()
    volK=better_volume(K,try_hard=True)
    volF=better_volume(F,try_hard=True)
    if volK!=0:
        if volF!=0:
            if volF.overlaps(volK)==False:
                K.dehn_fill(K.homological_longitude())
                F.dehn_fill(K.homological_longitude())
                if search_for_combinatorial_equivalence(K,F,index_M=2,index_N=2)==True:
                    verified_friends.append([knot,PD])
                
unclear_friends=[x for x in friends if x not in verified_friends]

print('Verified friends:',len(verified_friends))
print('Unclear friends:',len(unclear_friends))
print('time taken:',time.time()-start_time,'seconds')

Verified friends: 2050
Unclear friends: 0
time taken: 5.587981939315796 seconds
